### Loose EDA

data: from https://zenodo.org/record/4060432#.X5ySHHhKgWo


Thesis: identify humans (voice), animals, instruments, water, door, car, explosion (boom, thunder)

3 Main CNN Models:

Spectrograms
MFCC Image Analysis
Mean-MFCC Coefficient Analysis

To do items:

Update:
1. model on image analysis of Mel Frequency Spectrogram (images)
2. model on analysis of coefficients of MFCC (unless it ends up making sense to analyze the image as well)


- create new columns of labels I will use (tags too long and many)

- extract MFCC of all files to add to dataframe

- consider need to trim audio files to make same length, OR adding length to short files (padding them?)

In [1]:
import pandas as pd
%matplotlib inline

In [2]:
dev_info = pd.read_json('data/FSD50K.metadata/dev_clips_info_FSD50K.json').T

In [3]:
eval_info = pd.read_json('data/FSD50K.metadata/eval_clips_info_FSD50K.json').T


In [5]:
dev_info = dev_info.sort_index()
dev_info['track_num'] = dev_info.index
dev_info['track_num'] = dev_info['track_num'].apply(lambda x: str(x) + '.wav')
dev_info.head()

,title,description,tags,license,uploader,track_num
63,RalfHutterWorking.wav,"Ralf Hutter from Kraftwerk saying ""Working on ...","[male, voice]",http://creativecommons.org/licenses/by/3.0/,fectoper,63.wav
136,keyboard-rhymtic.wav,Noise of an average logitech keyboard. Pretty ...,"[keyboard, rhythmic, tap, type]",http://creativecommons.org/licenses/by/3.0/,Anton,136.wav
137,keyboard-typing.wav,Noise of an average logitech keyboard. Pretty ...,"[computer, environmental-sounds-research, key,...",http://creativecommons.org/licenses/by/3.0/,Anton,137.wav
221,bell.wav,simple *ting* sound,[bell],http://creativecommons.org/publicdomain/zero/1.0/,Erratic,221.wav
236,BUSSES.aiff,Departing busses\r\n at Utrecht Central Railwa...,"[bus, depart, drive, station]",http://creativecommons.org/licenses/by-nc/3.0/,hanstimm,236.wav


In [7]:
# Nothing useful here, but why would they include this if useless?

# ratings_info = pd.read_json('FSD50K.metadata/pp_pnp_ratings_FSD50K.json')

In [8]:
# eval_info.sort_index(inplace=True)
# eval_info['track_num'] = eval_info.index
# eval_info['track_num'] = eval_info['track_num'].apply(lambda x: str(x) + '.wav')
# #eval_info.head()

In [6]:
dev_info['track_num'] = dev_info.index
dev_info['track_num'] = dev_info['track_num'].apply(lambda x: str(x) + '.wav')
#dev_info.head()

In [10]:
# unique_tags = []

# for tag_list in dev_info.tags:
#     for sub_tag in tag_list:
#         if sub_tag not in unique_tags:
#             unique_tags.append(sub_tag)

# len(unique_tags)

In [7]:
# print(unique_tags[:100])

# Building Labels Column with Lamdba Conditional Functions

In [8]:
dev_info.head()

,title,description,tags,license,uploader,track_num
63,RalfHutterWorking.wav,"Ralf Hutter from Kraftwerk saying ""Working on ...","[male, voice]",http://creativecommons.org/licenses/by/3.0/,fectoper,63.wav
136,keyboard-rhymtic.wav,Noise of an average logitech keyboard. Pretty ...,"[keyboard, rhythmic, tap, type]",http://creativecommons.org/licenses/by/3.0/,Anton,136.wav
137,keyboard-typing.wav,Noise of an average logitech keyboard. Pretty ...,"[computer, environmental-sounds-research, key,...",http://creativecommons.org/licenses/by/3.0/,Anton,137.wav
221,bell.wav,simple *ting* sound,[bell],http://creativecommons.org/publicdomain/zero/1.0/,Erratic,221.wav
236,BUSSES.aiff,Departing busses\r\n at Utrecht Central Railwa...,"[bus, depart, drive, station]",http://creativecommons.org/licenses/by-nc/3.0/,hanstimm,236.wav


In [30]:
human_words = ['human', 'man', 'male', 'voice','woman','children',
               'boy', 'girl', 'talk', 'kids','announcement', 'announcer',
              'conversations', 'people', 'singing', 'vocals', 'chant']

comedy_words = ['comedy', 'laughing', 'laughs', 'laugh']

foley_words = ['keyboard', 'keys', 'coin', 'money', 'scratch', 'coins', 
               'crackle', 'eating', 'click', 'zipper', 'tools', 'foley', 'breath', 'cough']

baby_words = ['crying','baby', 'scream']

animal_words = ['dog', 'dogs', 'cat', 'bark', 'purr', 'frogs','frog', 'cricket', 
                'crickets', 'birds', 'bird', 'warbler', 'insects','birdsong',
               'bee', 'insect','pigs', 'pig','animal-sounds','grasshopper','bugs',
               'ducks', 'rooster', 'owl', 'monkey', 'animals', 'cattle',
               'hen', 'hens' 'fart', 'flatulence', 'animal']

music_words = ['instrument', 'guitar', 'piano', 'bell', 'shaker', 
               'sax', 'clarinet', 'riff', 'single-note', 'chord',
              'electric-piano', 'bass', 'cello', 'organ', 'violin', 'viola', 
               'double-bass', 'classical-guitar', 'chordophone',
               'transverse-flute','bassoon', 'flute', 'trumpet', 'harmonica',
              'music', 'brass', 'saxophone', 'orchestra', 'drumloop']

percussion_words = ['drums', 'drum', 'bass-drum', 'percussion', 
                    'cymbal' ,'hi-hat','tambourine''xylophone']

nature_words = ['water', 'rain', 'wind', 'ocean', 'waterfall', 'peeing',
                'whoosh', 'nature', 'faucet', 'boiling', 'dripping']

machinery_words = ['car', 'engine', 'bus', 'driving', 'drive', 'mixer',
                   'machine', 'drill','blender', 'blendering', 'train', 'metro']

ambient_words = ['environmental-sounds-research','applause', 'clap', 'crowd']

door_words = ['door', 'opening', 'closing', 'drawer', 'shut', 'close', 'gate', 'doorslam']

fire_words = ['lighter','fire', 'spark', 'explosion', 'glass', 'thunder', 
              'lightning', 'gun', 'crack','burning', 'bang', 'banging', 'hit',
             'beep', 'beeping', 'kettle', 'alarm', 'phone']

movement_words = ['footsteps', 'clop', 'walk']

# combine foley with household
household_words = ['cooking', 'pot', 'pan', 'slicing', 'cutting', 'cut', 
                   'sharpening', 'clock', 'ticking', 'paper', 'camera', 'metal']

'dance'
'electro'
'bells'
'helicopter'
'whisper'
'chords'
'didgeridoo'
'snare'
'groovy'
'world-music'
'trains'
'motorcycle'

def cat_labeler(tag_list):
    if set(human_words).intersection(tag_list):
        return 'human'
    elif set(comedy_words).intersection(tag_list):
        return 'comedy'
    elif set(door_words).intersection(tag_list):
        return 'door'  
    elif set(baby_words).intersection(tag_list):
        return 'baby'
    elif set(animal_words).intersection(tag_list):
        return 'animal'    
    elif set(music_words).intersection(tag_list):
        return 'music'
    elif set(percussion_words).intersection(tag_list):
        return 'percussion'
    elif set(machinery_words).intersection(tag_list):
        return 'machines'
    elif set(ambient_words).intersection(tag_list):
        return 'ambient'
    elif set(fire_words).intersection(tag_list):
        return 'fire'   
    elif set(movement_words).intersection(tag_list):
        return 'movement'
    elif set(household_words).intersection(tag_list):
        return 'household'
    elif set(foley_words).intersection(tag_list):
        return 'foley'  
    elif set(nature_words).intersection(tag_list):
        return 'nature'
    else:
        return 'other'
    
dev_info['labels'] = dev_info['tags'].apply(cat_labeler)    

In [31]:
for row in dev_info[dev_info.labels == 'other']['tags']:
    for tag in row:
        if tag == 'loop':
            print(row)

['133bpm', 'drumloop', 'electro', 'loop']
['115bpm', 'dance', 'drumloop', 'electronic', 'loop', 'weird']
['115bpm', 'dance', 'drumloop', 'electronic', 'loop', 'weird']
['115bpm', 'dance', 'drumloop', 'electronic', 'loop', 'weird']
['140-bpm', 'drumloop', 'electro', 'loop']
['140-bpm', 'drumloop', 'electro', 'loop']
['140-bpm', 'drumloop', 'electro', 'loop']
['acoustic', 'drumloop', 'loop']
['acoustic', 'drumloop', 'loop']
['acoustic', 'drumloop', 'loop']
['acoustic', 'drumloop', 'loop']
['acoustic', 'drumloop', 'loop']
['acoustic', 'drumloop', 'loop']
['bells', 'delay', 'loop', 'remix']
['120bpm', 'drumloop', 'electro', 'electronic', 'loop', 'rhythmic']
['120bpm', 'drumloop', 'electro', 'electronic', 'loop', 'rhythmic']
['120bpm', 'beat', 'binaural', 'drumloop', 'loop', 'rhythm', 'rhythmic']
['100bpm', 'dance', 'didgeridoo', 'didjeridu', 'fast', 'loop', 'rhythm', 'wild']
['bells', 'campanology', 'church', 'field-recording', 'loop', 'peal', 'pealing', 'repeat', 'ring', 'ringing']
['budd

In [ ]:
'dance'
'electro'
'bells'
'helicopter'
'whisper'
'chords'
'didgeridoo'
'snare'
'groovy'
'world-music'
'trains'
'motorcycle'

In [18]:
# dictionary for remaining tags

remaining_tags_dict = {}

for row in dev_info[dev_info.labels == 'other']['tags']:
    for tag in row:
        if tag not in remaining_tags_dict:
            remaining_tags_dict[tag] =1
        else:
            remaining_tags_dict[tag] += 1  


In [19]:
remaining_tags_series = pd.Series(remaining_tags_dict)
remaining_tags_series.sort_values(ascending=False).iloc[:20]

field-recording    487
metal              213
camera             172
noise              139
sound              123
loop               116
wood               115
owi                109
squeak             109
burp               103
samples             99
speech              92
plastic             89
shutter             88
toilet              85
metallic            84
bicycle             82
hammer              82
flush               80
horse               79
dtype: int64

In [20]:
dev_info.labels.value_counts()

music         10124
other          5416
human          4445
fire           3865
animal         3268
percussion     2489
foley          2281
machines       2258
nature         2232
door           1673
household      1058
ambient         801
movement        412
comedy          399
baby            245
Name: labels, dtype: int64

In [25]:
# dev_info[dev_info.labels == 'other'].labels
dev_info.labels.value_counts()

music         10124
other          5416
human          4445
fire           3865
animal         3268
percussion     2489
foley          2281
machines       2258
nature         2232
door           1673
household      1058
ambient         801
movement        412
comedy          399
baby            245
Name: labels, dtype: int64

In [32]:
dev_info.head()

,title,description,tags,license,uploader,track_num,labels
63,RalfHutterWorking.wav,"Ralf Hutter from Kraftwerk saying ""Working on ...","[male, voice]",http://creativecommons.org/licenses/by/3.0/,fectoper,63.wav,human
136,keyboard-rhymtic.wav,Noise of an average logitech keyboard. Pretty ...,"[keyboard, rhythmic, tap, type]",http://creativecommons.org/licenses/by/3.0/,Anton,136.wav,foley
137,keyboard-typing.wav,Noise of an average logitech keyboard. Pretty ...,"[computer, environmental-sounds-research, key,...",http://creativecommons.org/licenses/by/3.0/,Anton,137.wav,ambient
221,bell.wav,simple *ting* sound,[bell],http://creativecommons.org/publicdomain/zero/1.0/,Erratic,221.wav,music
236,BUSSES.aiff,Departing busses\r\n at Utrecht Central Railwa...,"[bus, depart, drive, station]",http://creativecommons.org/licenses/by-nc/3.0/,hanstimm,236.wav,machines


# Analysis of Labelling as of now:
#### of 50,000 audio files, only ~5,000 are not labelled (pretty good)(needs some more work)
#### need to review contents of each category, maybe some mislabelling (also order of 'elif' statements for binning files)
####  review best methods for multiclassification problems (is ten a reasonable number of categories, is more okay?) 

# Random EDA for labelling

In [101]:
unique_first_tags = []

for row in dev_info.tags:
    for tag in row:
        if tag not in dict_counter:
            dict_counter[tag] =1
        else:
            dict_counter[tag] += 1            

In [13]:
len(unique_first_tags)

4277

In [14]:
unique_first_tags[:5]

['male', 'keyboard', 'computer', 'bell', 'bus']

In [15]:
unique_second_tags = []

for tag_list in dev_info.tags:
    if len(tag_list) >1:
        if tag_list[1] not in unique_second_tags:
            unique_second_tags.append(tag_list[1])
            
            

In [16]:
unique_second_tags[:5]

['voice', 'rhythmic', 'environmental-sounds-research', 'depart', 'engine']

In [17]:
all_firsts = []

for i in dev_info.tags:
    all_firsts.append(i[0])

In [18]:
len(all_firsts)

40966

In [19]:
len(set(all_firsts))

4277

In [20]:
all_firsts_series = pd.Series(all_firsts)
all_firsts_series.value_counts()[:5]

In [22]:
# for i in dev_info.tags:
#     if i[0] == 'guitar':
#         if len(i) > 1:
#             print(i[1])

In [23]:
counter = 0

for i in dev_info.tags: 
    if 'female' in i:
        counter +=1

print(counter)        

1159


In [24]:
counter = 0

for i in dev_info.tags: 
    if 'male' in i:
        counter +=1

print(counter)

1045


In [25]:
# prints all tags for all files

# for i in dev_info.tags: 
#     if 'nature' in i:
#         print(i)

In [26]:
dict_counter = {}

for row in dev_info.tags:
    for tag in row:
        if tag not in dict_counter:
            dict_counter[tag] =1
        else:
            dict_counter[tag] += 1

In [27]:
#prints counter for all tags

#dict_counter

In [61]:
instrument_df = pd.DataFrame.from_dict(dict_counter, orient='index').sort_values(by=0, ascending= False)

In [64]:
instrument_df.iloc[:20]

,0
multisample,5564
single-note,3768
field-recording,3490
good-sounds,3081
neumann-u87,3080
voice,2511
water,2167
percussion,1728
guitar,1606
drum,1434


In [30]:
dev_info.iloc[100:105]

,title,description,tags,license,uploader,track_num
2081,CP_set2.wav,drumatic set 2,[clap],http://creativecommons.org/licenses/sampling+/...,opm,2081.wav
2083,CP_set4.wav,drumatic set 4,[clap],http://creativecommons.org/licenses/sampling+/...,opm,2083.wav
2139,breathehope - kitchen - 13.wav,From my Kitchen,"[beat, field-recording, kitchen, metal]",http://creativecommons.org/licenses/sampling+/...,smallsushi,2139.wav
2167,laundromat01.wav,"droning bass sound, recorded to minidisc in th...","[bass, drone, field-recording, machinery]",http://creativecommons.org/licenses/sampling+/...,edwin_p_manchester,2167.wav
2215,rustle1.wav,wind in a stove pipe (different again),"[field-recording, hiss, noise, rustle, stove-p...",http://creativecommons.org/licenses/by/3.0/,Andrew Duke,2215.wav
